In [ ]:
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

ImportError: cannot import name 'load_metric' from 'datasets' (/home/lcorbucci/FLTemplate/.venv/lib/python3.12/site-packages/datasets/__init__.py)

In [2]:
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 13662.23it/s]
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# load dummy dataset and read soundfiles
ds = load_dataset("SALT-NLP/speech_fairness", split="train")

In [4]:
ds

Dataset({
    features: ['audio', 'transcription', 'age', 'gender', 'first_language', 'socioeconomic_bkgd', 'ethnicity'],
    num_rows: 26471
})

In [5]:
# split the dataset into train, test and validation
ds = ds.train_test_split(test_size=0.2)
ds["validation"] = ds["train"].train_test_split(test_size=0.2)
ds["train"] = ds["validation"]["train"]
ds["validation"] = ds["validation"]["test"]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription', 'age', 'gender', 'first_language', 'socioeconomic_bkgd', 'ethnicity'],
        num_rows: 16940
    })
    test: Dataset({
        features: ['audio', 'transcription', 'age', 'gender', 'first_language', 'socioeconomic_bkgd', 'ethnicity'],
        num_rows: 5295
    })
    validation: Dataset({
        features: ['audio', 'transcription', 'age', 'gender', 'first_language', 'socioeconomic_bkgd', 'ethnicity'],
        num_rows: 4236
    })
})

##  Create Wav2Vec2CTCTokenizer 

In [7]:
ds = ds.remove_columns(["age", "first_language", "socioeconomic_bkgd"])

In [8]:

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(ds["train"].remove_columns(["audio"]))

,transcription,gender,ethnicity
0,hey facebook no reply to next message thread,female,Black or African American
1,hey facebook share my photo to instagram,female,"Asian, South Asian or Asian American"
2,hey facebook can you take a picture,female,Black or African American
3,hey facebook show a video of the graduation party to my close friends on my instagram story,female,Black or African American
4,fast-forward one track,female,"Asian, South Asian or Asian American"
5,hey hey facebook can you turn up my noise by forty percent,female,"Hispanic, Latino, or Spanish"
6,play music by ll cool j,female,"Native American, American Indian, or Alaska Native"
7,send reply text message to adam happy kwanzaa to you too my brother,male,"Native American, American Indian, or Alaska Native"
8,hey sam and i will be coming at five pm to pick you up and head to the movies see you then,female,"Asian, South Asian or Asian American"
9,hi facebook hide my on line status on whatsapp,male,Black or African American


In [9]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    if batch["transcription"] is not None:
        batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower()
    else:
        batch["transcription"] = ""
    return batch

ds = ds.map(remove_special_characters)

<>:2: SyntaxWarning: invalid escape sequence '\,'
<>:2: SyntaxWarning: invalid escape sequence '\,'
/tmp/ipykernel_1634931/2137285444.py:2: SyntaxWarning: invalid escape sequence '\,'
  chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
Map: 100%|██████████| 4236/4236 [00:01<00:00, 3020.29 examples/s]


In [10]:
show_random_elements(ds["train"].remove_columns(["audio"]))

,transcription,gender,ethnicity
0,message alexander with i finally got to read julie klassen's the apothecary's daughter,female,"Asian, South Asian or Asian American"
1,where are you,female,"Native American, American Indian, or Alaska Native"
2,do you know that i start a business with zero cent with the help of my friend,male,Black or African American
3,hey facebook can you read me the incoming text from my group of friends on sms,female,"Native American, American Indian, or Alaska Native"
4,hey facebook please remove me from group message on whatsapp,female,Native Hawaiian or Other Pacific Islander
5,hey facebook share this video to my mom in viber,female,"Asian, South Asian or Asian American"
6,hey facebook send a message to nancy ask her if she is okay,female,White
7,send a message to my friend saying i am watching this tv show you should watch it,male,"Hispanic, Latino, or Spanish"
8,reply text to ethan did you hear what i heard last week question mark i now understand what was going on with amy period i think she is looking for other work period,female,White
9,hey facebook can you dial hailey my daughter,female,"Native American, American Indian, or Alaska Native"


In [11]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = ds.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=ds.column_names["train"])

Map: 100%|██████████| 4236/4236 [00:00<00:00, 688166.08 examples/s]


In [12]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'e': 0,
 'ó': 1,
 'x': 2,
 'g': 3,
 'q': 4,
 'u': 5,
 'm': 6,
 'p': 7,
 't': 8,
 'r': 9,
 'o': 10,
 's': 11,
 '\\': 12,
 'a': 13,
 'é': 14,
 'z': 15,
 'ñ': 16,
 'n': 17,
 'l': 18,
 ' ': 19,
 'w': 20,
 'f': 21,
 'b': 22,
 'í': 23,
 '́': 24,
 'd': 25,
 '\u200e': 26,
 'k': 27,
 'i': 28,
 '’': 29,
 'y': 30,
 'h': 31,
 'v': 32,
 'c': 33,
 'j': 34,
 "'": 35}

In [13]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [14]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

38


In [ ]:
import json
with open('../data/speech_fairness/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [18]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("../data/speech_fairness/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

##  Create Wav2Vec2 Feature Extractor 

In [19]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [20]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


## Preprocess Data

In [25]:
print(ds["train"][0]["audio"]["path"])

13e53232266aac1a8de26fc27cf2083f.wav


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(ds["train"]))

print(ds["train"][rand_int]["transcription"])
ipd.Audio(data=np.asarray(ds["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

hey facebook make a new instagram story to my close friends open a photo


In [32]:
rand_int = random.randint(0, len(ds["train"]))

print("Target text:", ds["train"][rand_int]["transcription"])
print("Input array shape:", np.asarray(ds["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", ds["train"][rand_int]["audio"]["sampling_rate"])


Target text: hey facebook i wanna take a picture
Input array shape: (45733,)
Sampling rate: 16000


In [37]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [38]:
ds = ds.map(prepare_dataset, remove_columns=ds.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/16940 [00:00<?, ? examples/s]/home/lcorbucci/FLTemplate/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/lcorbucci/FLTemplate/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/lcorbucci/FLTemplate/.venv/lib/python3.12/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:180: UserWarning: `as_target_processor` is d

# Setup trainer

In [44]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    def __init__(
        self,
        processor, 
        padding = True):
            self.processor: Wav2Vec2Processor = processor
            self.padding: Union[bool, str] = padding
            self.max_length: Optional[int] = None
            self.max_length_labels: Optional[int] = None
            self.pad_to_multiple_of: Optional[int] = None
            self.pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [45]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [52]:
import evaluate

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [53]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

/home/lcorbucci/FLTemplate/.venv/lib/python3.12/site-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
model.freeze_feature_extractor()

In [64]:
import accelerate

In [68]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    save_steps=500,
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    warmup_steps=500,
    logging_dir="./logs",
    fp16=True,  # Mixed precision for faster training on GPUs
)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)


NameError: name 'training_args' is not defined

In [ ]:
trainer.train()

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-colab")

# Evaluation

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)


In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)


In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))
